In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [4]:
%pip install "stable-baselines3[extra]"

  Using cached stable_baselines3-2.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached gymnasium-1.2.1-py3-none-any.whl.metadata (10.0 kB)
  Using cached numpy-2.3.3-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached torch-2.8.0-cp311-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pandas-2.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached matplotlib-3.10.6-cp311-cp311-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached pygame-2.6.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached psutil-7.1.0-cp36-abi3-macosx_11_0_arm64.whl.metadata (23 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached rich-14.1.0-py3-none-any.whl.metadata (18 kB)
  Using cached ale_py-0.11.2-cp311-cp3

In [7]:
import os
import gymnasium as gym 
from stable_baselines3 import PPO # Proximal Policy Optimization
from stable_baselines3.common.vec_env import DummyVecEnv # Wrapper to work with vectorized environments
from stable_baselines3.common.evaluation import evaluate_policy # Helper to evaluate the agent

# 2. Load Environment

In [2]:
environment_name = "CartPole-v0"

In [3]:
env = gym.make(environment_name)

/Users/sahand/ReinforcementLearningCourse/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [5]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:13.0
Episode:2 Score:63.0
Episode:3 Score:29.0
Episode:4 Score:22.0
Episode:5 Score:18.0


/Users/sahand/ReinforcementLearningCourse/.venv/lib/python3.11/site-packages/gymnasium/envs/classic_control/cartpole.py:250: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [ ]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

In [ ]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

# 3. Train an RL Model

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

In [ ]:
model.learn(total_timesteps=20000)

# 4. Save and Reload Model

In [ ]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [ ]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load('PPO_model', env=env)

# 4. Evaluation

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 5. Test Model

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

In [ ]:
env.close()

# 6. Viewing Logs in Tensorboard

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [ ]:
!tensorboard --logdir={training_log_path}

# 7. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()